In [1]:
import pickle
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from datasets import load_metric, GeneratorBasedBuilder, load_dataset, load_from_disk
import numpy as np

import json

from typing import List

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
def open_file(path:str) -> dict:
    file = open(path, 'rb')
    pickle_dict = pickle.load(file)
    file.close()

    return pickle_dict

In [4]:
TRAIN_PKL = r'C:\Users\beatriz.maia\Documents\nlp_studies\covid19_ner\ml4nlp-covid19ner\train.pkl'
TEST_PKL = r'C:\Users\beatriz.maia\Documents\nlp_studies\covid19_ner\ml4nlp-covid19ner\test.pkl'
VAL_PKL = r'C:\Users\beatriz.maia\Documents\nlp_studies\covid19_ner\ml4nlp-covid19ner\val.pkl'

In [5]:
train_dict = open_file(TRAIN_PKL)
test_dict = open_file(TEST_PKL)
val_dict = open_file(VAL_PKL)

In [6]:
print("train keys:", train_dict.keys())
print("test keys:", test_dict.keys())
print("val keys:", val_dict.keys())

train keys: dict_keys(['id', 'word_seq', 'tag_seq'])
test keys: dict_keys(['id', 'word_seq'])
val keys: dict_keys(['id', 'word_seq', 'tag_seq'])


In [7]:
print("train len:", len(train_dict['id']))
print("test len:",  len(test_dict['id']))
print("val len:",  len(val_dict['id']))

train len: 23600
test len: 2950
val len: 2950


Divisao de treino/test/validacao: 80/10/10

In [8]:
# getting unique ner_tags
ner_tags = set()

for value_array in train_dict['tag_seq']:
    for word in value_array:
        ner_tags.add(word)

for value_array in val_dict['tag_seq']:
    for word in value_array:
        ner_tags.add(word)

ner_tags

{'ANATOMICAL_STRUCTURE',
 'ARCHAEON',
 'BACTERIUM',
 'BODY_PART_ORGAN_OR_ORGAN_COMPONENT',
 'BODY_SUBSTANCE',
 'CARDINAL',
 'CELL',
 'CELL_COMPONENT',
 'CELL_FUNCTION',
 'CELL_OR_MOLECULAR_DYSFUNCTION',
 'CHEMICAL',
 'CORONAVIRUS',
 'DAILY_OR_RECREATIONAL_ACTIVITY',
 'DATE',
 'DIAGNOSTIC_PROCEDURE',
 'DISEASE_OR_SYNDROME',
 'EDUCATIONAL_ACTIVITY',
 'EUKARYOTE',
 'EVENT',
 'EVOLUTION',
 'EXPERIMENTAL_MODEL_OF_DISEASE',
 'FAC',
 'FOOD',
 'GENE_OR_GENOME',
 'GOVERNMENTAL_OR_REGULATORY_ACTIVITY',
 'GPE',
 'GROUP',
 'GROUP_ATTRIBUTE',
 'HUMAN-CAUSED_PHENOMENON_OR_PROCESS',
 'IMMUNE_RESPONSE',
 'INDIVIDUAL_BEHAVIOR',
 'INJURY_OR_POISONING',
 'LABORATORY_OR_TEST_RESULT',
 'LABORATORY_PROCEDURE',
 'LANGUAGE',
 'LAW',
 'LIVESTOCK',
 'LOC',
 'MACHINE_ACTIVITY',
 'MATERIAL',
 'MOLECULAR_FUNCTION',
 'MONEY',
 'NORP',
 'O',
 'ORDINAL',
 'ORG',
 'ORGANISM',
 'ORGAN_OR_TISSUE_FUNCTION',
 'PERCENT',
 'PERSON',
 'PHYSICAL_SCIENCE',
 'PRODUCT',
 'QUANTITY',
 'RESEARCH_ACTIVITY',
 'SIGN_OR_SYMPTOM',
 'SO

In [9]:
ner_tags_id = dict()
ner_tags_values = dict()
for i, value in enumerate(ner_tags):
    ner_tags_id[i] = value
    ner_tags_values[value] = i

In [10]:
def dict_to_array(dicitionary: dict, ner_tags_values:dict=None) -> list:
    id_values = dicitionary['id']
    word_values = dicitionary['word_seq']

    array = []
    if 'tag_seq' in dicitionary.keys():
        tag_values = dicitionary['tag_seq']

        for id, word, tag in zip(id_values, word_values, tag_values):
            ner_tags = [ner_tags_values[value] for value in tag]
            aux = {'id': id, 
                   'tokens': word,
                   'ner_tags': ner_tags}
            array.append(aux)
    else:
        for id, word in zip(id_values, word_values):
            aux = {'id': id, 
                   'tokens': word}
            array.append(aux)
    
    return array


In [11]:
train_array = dict_to_array(train_dict, ner_tags_values)
val_array = dict_to_array(val_dict, ner_tags_values)
test_array = dict_to_array(test_dict)

In [12]:
def write_file(path:str, data:List[dict]) -> None:
    with open(path, "w") as final:
        for item in data:
            json.dump(item, final)

In [13]:
write_file(r'train.json', train_array)
write_file(r'test.json', test_array)
write_file(r'val.json', val_array)

In [14]:
data_files = {"train": "train.json", "test": "test.json", 'val':'val.json'}

In [15]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-cased/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1131)')))


OSError: Can't load config for 'bert-base-cased'. Make sure that:

- 'bert-base-cased' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'bert-base-cased' is the correct path to a directory containing a config.json file



In [ ]:
train_encodings = tokenizer(train_array, truncation=True, padding=True)

NameError: name 'tokenizer' is not defined

In [ ]:
class COVID19Ner(GeneratorBasedBuilder):
    TRAIN_PKL = r'C:\Users\beatriz.maia\Documents\nlp_studies\covid19_ner\ml4nlp-covid19ner\train.pkl'
    TEST_PKL = r'C:\Users\beatriz.maia\Documents\nlp_studies\covid19_ner\ml4nlp-covid19ner\test.pkl'
    VAL_PKL = r'C:\Users\beatriz.maia\Documents\nlp_studies\covid19_ner\ml4nlp-covid19ner\val.pkl'

    def __init__self(self,
                 *args,
                 url="https://www.kaggle.com/competitions/ml4nlp-covid19ner/data?select=val_labels.csv",
                 ner_tags=("company", "date", "address", "total", "O"),
                 **kwargs):
    

SyntaxError: unexpected EOF while parsing (1068575552.py, line 11)

In [ ]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-cased/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1131)')))


OSError: Can't load config for 'bert-base-cased'. Make sure that:

- 'bert-base-cased' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'bert-base-cased' is the correct path to a directory containing a config.json file

